# Data Cleaning with Pandas

In this notebook we'll go through a few basic data cleaning steps that should be performed on all new datasets where necessary.

We'll go through the process with both the `orders` and `orderlines` datasets. You can then practice these skills by cleaning the `products` dataset yourself

In [21]:
import pandas as pd

In [22]:
# orders.csv
url = "https://drive.google.com/file/d/1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders = pd.read_csv(path)

# orderlines.csv
url = "https://drive.google.com/file/d/1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines = pd.read_csv(path)

One of the best ways to begin data cleaning is by exploring using `.info()`. This will tell us:
* The shape of the DataFrame
* The names of the columns
* If there are any missing values
* The datatypes of the columns

By exploring the missing values and correcting any incorrect datatypes, we often come across inconsistencies in our data.

Beyond this, we should also have a **check for any duplicate rows**. 

Let's first deal with the duplicates, as it's nice and easy, then we'll explore what `.info()` has to tell us.

## 1.&nbsp; Duplicates
We can check for duplicates using the pandas [.duplicated()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html) method. 

We can then delete these rows, if we wish, using [.drop_duplicates()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

In [23]:
# orders
orders.duplicated().sum()

0

In [24]:
# orderlines
orderlines.duplicated().sum()

0

We have no duplicate rows in either DataFrame. Easy, there is no problem to solve. Normally though, if there were some duplicates, we'd drop the extra rows.

# 2.&nbsp; `.info()`

In [25]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


* `total_paid` has 5 missing values
* `created_date` should become datetime datatype

In [26]:
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


* `date` should be a datetime datatype
* `unit_price` should be a float datatype

## 3.&nbsp; Missing values

### 3.1.&nbsp; Orders
* `total_paid` has 5 missing values

In [27]:
print(f"5 missing values represents {((orders.total_paid.isna().sum() / orders.shape[0])*100).round(5)}% of the rows in our DataFrame")

5 missing values represents 0.0022% of the rows in our DataFrame


A quick way to find out a percentage here, if you don't need to print out a sentence for yourself/students/colleagues is `.value_count(normalize=True)`

In [28]:
orders.total_paid.isna().value_counts(normalize=True)

False    0.999978
True     0.000022
Name: total_paid, dtype: float64

As there is such a tiny amount of missing values, we will simply delete these rows, as we have enough data without them.

In [29]:
orders = orders.loc[~orders.total_paid.isna(), :]

Should you have a significant number of missing values in the future, you have a choice: 
+ you can impute the values
+ you can take the values from other DataFrames, if they are present there
+ you can delete the values
+ or any number of other creative solutions

Please, always consider how much time you have on your project, and what impact your method of choice will have on your final assesment.

### 3.2.&nbsp; Orderlines
There are no missing values in `orderlines`

## 4.&nbsp; Datatypes

### 4.1.&nbsp; Orders
* `created_date` should become datetime datatype

In [30]:
orders["created_date"] = pd.to_datetime(orders["created_date"])

In [31]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226904 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      226904 non-null  int64         
 1   created_date  226904 non-null  datetime64[ns]
 2   total_paid    226904 non-null  float64       
 3   state         226904 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 8.7+ MB


### 4.1.&nbsp; Orderlines
* `date` should be a datetime datatype
* `unit_price` should be a float datatype

#### 4.1.1.&nbsp; `date`

In [32]:
orderlines["date"] = pd.to_datetime(orderlines["date"])

In [33]:
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  object        
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 15.7+ MB


#### 4.1.2.&nbsp;`unit_price`

In [34]:
orderlines["unit_price"] = pd.to_numeric(orderlines["unit_price"])

ValueError: Unable to parse string "1.137.99" at position 6

As you can see when we try to convert `unit_price` to a numerical datatype, we receive a `ValueError` telling us that pandas doesn't understand the number `1.137.99`. This is probably because numbers cannot have 2 decimal points. Let's see if there are any other numbers like this.

In [ ]:
orderlines.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()

In [ ]:
#hulya
# extract numeric values (without dots)
#extract from Alpha numeric column just numeric values
df_products=products
df_products
df_products['price_extracted']=df_products.price.str.extract('([-+]?\d*\.?\d+)')
df_products['price_extracted']=df_products['price_extracted'].astype(float)
df_products.dtypes
df_products.head(10)

Looks like over 36000 rows in `orderlines` are affected by this problem. Let's work out how much that is as a percentage of our total data.

In [ ]:
# this line calulates the percentage of rows with the 2 dot problem
two_dot_percentage = ((orderlines.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()[1] / orderlines.shape[0])*100).round(2)
print(f"The 2 dot problem represents {two_dot_percentage}% of the rows in our DataFrame")

The 2 dot problem represents 12.3% of the rows in our DataFrame


This is a bit of a tricky decision as 12.3% is a significant amount of our data... and we might even end up losing a larger portion of our data than this too. For the moment we will delete the rows as we only have 2 weeks for this project and I'd like some quick, accurate results to show. If we have time at the end, we can come back and investigate this problem further, maybe there's a solution?

Each row of `orderlines` represents a product in an order. For example, if order number 175 contained 3 seperate products, then order 175 would have 3 rows in `orderlines`, one row for each of the products. If 2 of those products have 'normal' prices (14.99, 15.85) and 1 has a price with 2 decimal points (1.137.99), we need to remove the whole order and not just the affected row. If we only remove the row with 2 decimal places then any later analysis about products and prices could be misleading.

We therefore need to find the order numbers associated with the rows that have 2 decimal points, and then remove all the associated rows.

In [ ]:
# this line of code will return the order ids that have the 2 dot problem
two_dot_order_ids_list = orderlines.loc[orderlines.unit_price.str.contains("\d+\.\d+\.\d+"), "id_order"]


In [ ]:
# this line of code will remove the rows that have the 2 dot problem
orderlines = orderlines.loc[~orderlines.id_order.isin(two_dot_order_ids_list)]

In [ ]:
orders.head(10)

,order_id,created_date,total_paid,state
0,241319,2017-01-02 13:35:40,44.99,Cancelled
1,241423,2017-11-06 13:10:02,136.15,Completed
2,242832,2017-12-31 17:40:03,15.76,Completed
3,243330,2017-02-16 10:59:38,84.98,Completed
4,243784,2017-11-24 13:35:19,157.86,Cancelled
5,245275,2017-06-28 11:35:37,149.00,Completed
6,245595,2017-01-21 12:52:47,112.97,Completed
7,245851,2017-04-04 20:58:21,79.99,Pending
8,245941,2017-01-01 10:32:23,183.52,Completed
9,246018,2017-02-13 23:02:57,211.95,Completed


In [18]:
orderlines.head(10)

,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38
5,1119114,295310,0,10,WDT0249,231.79,2017-01-01 01:14:27
6,1119115,299544,0,1,APP1582,1.137.99,2017-01-01 01:17:21
7,1119116,299545,0,1,OWC0100,47.49,2017-01-01 01:46:16
8,1119119,299546,0,1,IOT0014,18.99,2017-01-01 01:50:34
9,1119120,295347,0,1,APP0700,72.19,2017-01-01 01:54:11


In [36]:
# column unit_price has following values(18.99, 399.00, 474.05, 1.137.99, , 2.345.23). Remove first dot only  in the values that have two dots. Keep dots in values that have only one dot.


replace_dots = orderlines.unit_price.copy().str.replace("\.", "", 1)
# orderlines.unit_price.str.replace("\.", "", 1).str.replace(",", ".").astype(float)
replace_dots.head(20)

/var/folders/fv/j2glptx96zv4jxc02f5d5ff40000gn/T/ipykernel_56730/1517171475.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  replace_dots = orderlines.unit_price.copy().str.replace("\.", "", 1)


0        1899
1       39900
2       47405
3        6839
4        2374
5       23179
6     1137.99
7        4749
8        1899
9        7219
10       3514
11    2565.99
12      14914
13       1699
14       1999
15    3278.99
16       5984
17      25649
18       6090
19       3799
Name: unit_price, dtype: object

In [37]:
replace_with_float = orderlines.unit_price.copy().str.replace("\.", "", 1).str.replace(",", ".").astype(float)
replace_with_float

/var/folders/fv/j2glptx96zv4jxc02f5d5ff40000gn/T/ipykernel_56730/2061687964.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  replace_with_float = orderlines.unit_price.copy().str.replace("\.", "", 1).str.replace(",", ".").astype(float)


0          1899.0
1         39900.0
2         47405.0
3          6839.0
4          2374.0
           ...   
293978     4299.0
293979    14158.0
293980      999.0
293981     1999.0
293982     1399.0
Name: unit_price, Length: 293983, dtype: float64

In [38]:
# filter the rows that have 2 dots
# replace the first dot with empty string
filter_two_dots = orderlines.unit_price.str.contains("\d+\.\d+\.\d+")
orderlines.loc[filter_two_dots, "unit_price"] = orderlines.loc[filter_two_dots, "unit_price"].str.replace("\.", "", 1)
orderlines.unit_price.head(20)
# orderlines.loc[filter_two_dots, "unit_price"] = orderlines.loc[filter_two_dots, "unit_price"].str.replace(",", ".").astype(float)

/var/folders/fv/j2glptx96zv4jxc02f5d5ff40000gn/T/ipykernel_56730/2129408863.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  orderlines.loc[filter_two_dots, "unit_price"] = orderlines.loc[filter_two_dots, "unit_price"].str.replace("\.", "", 1)


0       18.99
1      399.00
2      474.05
3       68.39
4       23.74
5      231.79
6     1137.99
7       47.49
8       18.99
9       72.19
10      35.14
11    2565.99
12     149.14
13      16.99
14      19.99
15    3278.99
16      59.84
17     256.49
18      60.90
19      37.99
Name: unit_price, dtype: object

In [ ]:
orderlines.shape[0]

216250

We still have 216250 rows in orderlines to work with. This should be more than enough for our evaluation.

Now that all of the 2 decimal point prices have been removed, let's try again to convert the column `unit_price` to the correct datatype.

In [ ]:
orderlines["unit_price"] = pd.to_numeric(orderlines["unit_price"])

It worked perfectly

# Challenge: Clean the `products` DataFrame
Now it's your turn. Use the lessons you learnt above and clean the products DataFrame. You don't have to copy exactly what we did. Think about the consequences of your actions, sometimes it is ok to delete rows, other times you may wish to come up with more creative solutions.

In [ ]:
# products.csv
url = "https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)

In [ ]:
# your code here
products.head(60)

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364
5,APP0073,Apple Composite AV Cable iPhone and iPod white,IPhone and iPod AV Cable Dock to Composite Video.,45,420.003,0,1230
6,KIN0008,Mac Memory Kingston 1GB 667MHz DDR2 SO-DIMM,1GB RAM Mac mini and iMac (2006/07) MacBook Pr...,18.99,146.471,0,1364
7,KIN0009,Mac Memory Kingston 2GB 800MHz DDR2 SO-DIMM,2GB RAM iMac with Intel Core 2 Duo (Penryn).,36.99,274.694,0,1364
8,KIN0001-2,Mac memory Kingston 4GB (2x2GB) 667MHz DDR2 SO...,RAM 4GB (2x2GB) Mac mini and iMac (2006/07) Ma...,74,669.904,0,1364
9,APP0100,Apple Adapter Mini Display Port to VGA,Adapter Mini Display Port to VGA MacBook and M...,35,330.003,0,1325


### Look for Duplicates

0

In [ ]:
# your code here

### Look for Missing values


In [ ]:
# your code here

### Check / Change Data types

In [ ]:
# your code here

In [ ]:
#simge
two_dot_order_sku_list = products.loc[products.price.str.contains("\d+\.\d+\.\d+", na=False), "sku"]
products = products.loc[~products.sku.isin(two_dot_order_sku_list)]
products["price"] = pd.to_numeric(products["price"])

In [ ]:
orders.groupby("state").agg({"total_paid":"sum", "order_id":"count"}).sort_values(by="total_paid", ascending=False).head()

,total_paid,order_id
state,,
Shopping Basket,85912559.93,117809
Place Order,17087858.32,40883
Completed,15910079.24,46605
Cancelled,5545816.86,7233
Pending,4703300.72,14379
